
**Convolutional layers from scratch**

# 1st method:

# reference: 
https://wiseodd.github.io/techblog/2016/07/16/convnet-conv-layer/

and 

https://github.com/huyouare/CS231n/blob/master/assignment2/cs231n/im2col.py

In [ ]:
class Convolutional_Layers_Scratch(object):
  def __init__(self):
        pass
    
  def forward_cnn(self, X, F, b, stride=1, padding=1):
    """
    X:  D x C x H x W
    F:  NF x C x HF x WF
    b: F x 1

    D: the number of input
    C:  then number of channel
    H:  the height of input image
    W: the weight of input image

    NF: the number of the filter
    HF: the height of the filter
    WF: the width of the filter

    out_H = (H - HF + 2P)/S + 1
    """

    cache = F, b, stride, padding
    n_filter, d_filter, h_filter, w_filter = F.shape
    n_x, d_x, h_x, w_x = X.shape
    
    # make sure is integer
    assert (h_x - h_filter + 2*padding) % stride == 0
    assert (w_x - w_filter + 2*padding) % stride == 0
    h_out = (h_x - h_filter + 2*padding)/stride + 1
    w_out = (w_x - w_filter + 2*padding)/stride + 1

    h_out, w_out = int(h_out), int(w_out)

    X_col = self.im2col_indices(X, h_filter, w_filter, padding=padding, stride=stride)
    W_col = F.reshape(n_filter, -1)
    
    out = W_col @ X_col + b
    out = out.reshape(n_filter, h_out, w_out, n_x)
    out = out.transpose(3, 0, 1, 2)

    cache = (X, F, b, stride, padding, X_col)

    return out, cache
    
  def conv_backward(dout, cache):
    X, F, b, stride, padding, X_col = cache
    n_filter, d_filter, h_filter, w_filter = F.shape

    db = np.sum(dout, axis=(0, 2, 3))              # dout -> db
    db = db.reshape(n_filter, -1)

    dout_reshaped = dout.transpose(1, 2, 3, 0).reshape(n_filter, -1)
    dF = dout_reshaped @ X_col.T
    dF = dF.reshape(F.shape)

    F_reshape = F.reshape(n_filter, -1)
    dX_col = F_reshape.T @ dout_reshaped
    dX = col2im_indices(dX_col, X.shape, h_filter, w_filter, padding=padding, stride=stride)


    return dX, dF, db


  def im2col_indices(self,x, field_height, field_width, padding=1, stride=1):
      """ An implementation of im2col based on some fancy indexing """
      # Zero-pad the input
      p = padding
      x_padded = np.pad(x, ((0, 0), (0, 0), (p, p), (p, p)), mode='constant')

      k, i, j = get_im2col_indices(x.shape, field_height, field_width, padding,
                                  stride)

      cols = x_padded[:, k, i, j]
      C = x.shape[1]
      cols = cols.transpose(1, 2, 0).reshape(field_height * field_width * C, -1)
      return cols

  def get_im2col_indices(self, x_shape, field_height, field_width, padding=1, stride=1):
      # First figure out what the size of the output should be
      N, C, H, W = x_shape
      assert (H + 2 * padding - field_height) % stride == 0
      assert (W + 2 * padding - field_height) % stride == 0
      out_height = (H + 2 * padding - field_height) / stride + 1
      out_width = (W + 2 * padding - field_width) / stride + 1

      i0 = np.repeat(np.arange(field_height), field_width)
      i0 = np.tile(i0, C)
      i1 = stride * np.repeat(np.arange(out_height), out_width)
      j0 = np.tile(np.arange(field_width), field_height * C)
      j1 = stride * np.tile(np.arange(out_width), out_height)
      i = i0.reshape(-1, 1) + i1.reshape(1, -1)
      j = j0.reshape(-1, 1) + j1.reshape(1, -1)

      k = np.repeat(np.arange(C), field_height * field_width).reshape(-1, 1)

      return (k, i, j)


  def col2im_indices(self, cols, x_shape, field_height=3, field_width=3, padding=1,
                    stride=1):
    """ An implementation of col2im based on fancy indexing and np.add.at """
    N, C, H, W = x_shape
    H_padded, W_padded = H + 2 * padding, W + 2 * padding
    x_padded = np.zeros((N, C, H_padded, W_padded), dtype=cols.dtype)
    k, i, j = get_im2col_indices(x_shape, field_height, field_width, padding,
                                stride)
    cols_reshaped = cols.reshape(C * field_height * field_width, -1, N)
    cols_reshaped = cols_reshaped.transpose(2, 0, 1)
    np.add.at(x_padded, (slice(None), k, i, j), cols_reshaped)
    if padding == 0:
      return x_padded
    return x_padded[:, :, padding:-padding, padding:-padding]

#2nd method:

credit: 
 https://towardsdatascience.com/a-guide-to-convolutional-neural-networks-from-scratch-f1e3bfc3e2de
